# Update CDA mappings in MDB

In [1]:
from neo4j import GraphDatabase, basic_auth
from mdb_tools import NelsonMDB
from mdb_tools.mdb_tools import get_entity_type
from bento_meta.entity import Entity
from bento_meta.objects import Term, Concept, Predicate, Node, Edge, Property
import pandas as pd
import numpy as np
import csv
import ast
pd.set_option("display.max_rows", None)

In [2]:
# MDB sandbox
URL = "bolt://localhost:7687" # <URL for database>
USER = "neo4j" # <Username for database>
PASSWORD = "noble-use-dairy" # <Password for database>
mdbn = NelsonMDB(uri=URL, user=USER, password=PASSWORD)

In [3]:
# this file connects to tables in CDA documentation on mappings
# each sheet within the file is a different endpoint
input_path = "C:/Users/nelso/OneDrive - Georgetown University/School Stuff/Capstone/Mappings/"
input_file = "CDA_mappings_20220721.xlsx"
input_filepath = input_path + input_file

In [ ]:
def format_cda_mapping_csv(
    input_filepath: str,
    output_filepath: str = "",
):
    if not output_filepath:
        output_filepath = 
    endpoints = []
    mapping_dfs = []
    for sheet in pd.read_excel(input_filepath, None).keys():
        endpoint = sheet.split(" ")[0] # type: ignore
        endpoints.append(endpoint)
        df = pd.read_excel(input_filepath, sheet_name=sheet)
        df.name = endpoint
        mapping_dfs.append(df)
    models = list(col for col in mapping_dfs[0].columns if col != "field")

In [ ]:
output_path = input_path
output_file = input_file.split(".")[0] + "_clean.csv"
output_other_file = input_file.split(".")[0] + "_other.csv"
output_file_path = output_path + output_file
output_other_file_path = output_path + output_other_file

In [ ]:
def link_synonym_csv(
    csv_filepath: str,
):
    """
    Given csv file of synonymous entities, links them in MDB via Concept.

    Input CSV should be formatted so that each line contains the information to uniquely
    identify two synonymous entities.
    """

In [3]:
# this file connects to tables in CDA documentation on mappings
# each sheet within the file is a different encpoint
input_path = "C:/Users/nelso/OneDrive - Georgetown University/School Stuff/Capstone/Mappings/"
input_file = "CDA_mappings_20220721.xlsx"
input_file_path = input_path + input_file

endpoints = []
mapping_dfs = []
for sheet in pd.read_excel(input_file_path, None).keys():
    endpoint = sheet.split(" ")[0] # type: ignore
    endpoints.append(endpoint)
    df = pd.read_excel(input_file_path, sheet_name=sheet)
    df.name = endpoint
    mapping_dfs.append(df)
models = list(col for col in mapping_dfs[0].columns if col != "field")

In [5]:
mapping_dfs[0]

,field,GDC,PDC,IDC
0,id,files.cases.submitter_id,files.cases.case_submitter_id,files.PatientID
1,identifier.system,GDC,PDC,IDC
2,identifier.value,files.cases.submitter_id,files.cases.case_submitter_id,files.PatientID
3,species,Homo sapiens,files.cases.taxon,files.tcia_species
4,sex,files.cases.demographic.gender,files.cases.demographics.gender,NOT CURRENTLY MAPPED
5,race,files.cases.demographic.race,files.cases.demographics.race,NOT CURRENTLY MAPPED
6,ethnicity,files.cases.demographic.ethnicity,files.cases.demographics.ethnicity,NOT CURRENTLY MAPPED
7,days_to_birth,files.cases.demographic.days_to_birth,files.cases.demographics.days_to_birth,NOT CURRENTLY MAPPED
8,subject_associated_project,files.cases.project.project_id,files.cases.project_submitter_id,files.collection_id
9,vital_status,files.cases.demographic.vital_status,cases.demographics.vital_status,NOT CURRENTLY MAPPED


In [5]:
cda_mapping_df = pd.DataFrame(columns=["model_1", "prop_handle_1", "node_handle_1",
                                        "model_2", "prop_handle_2", "node_handle_2"])

for df in mapping_dfs:
    df_prop_handle_1 = np.tile(df.iloc[:, 0], len(models))
    df_model_1 = np.repeat("CDA", len(df_prop_handle_1))
    df_model_2 = np.repeat(models, len(df.iloc[:, 0]))
    df_node_handle_1 = np.repeat(df.name, len(df_prop_handle_1))
    node_handles_2_list = []
    for model in models:
        node_handles_2_list.extend(df[model].tolist())
    prop_handles_2_list = []
    for handle in node_handles_2_list:
        if not "." in handle or "{" in handle:
            prop_handles_2_list.append(handle)
        else:
            prop_handles_2_list.append(handle.split(".")[-1])
    
    formatted_df = pd.DataFrame({"model_1": df_model_1,
                                "prop_handle_1": df_prop_handle_1,
                                "node_handle_1": df_node_handle_1,
                                "model_2": df_model_2,
                                "prop_handle_2": prop_handles_2_list,
                                "node_handle_2": node_handles_2_list})
    cda_mapping_df = pd.concat([cda_mapping_df, formatted_df], ignore_index=True)

# remove unmapped entities    
cda_mapping_df = cda_mapping_df[cda_mapping_df.node_handle_2 != "NOT CURRENTLY MAPPED"]
cda_mapping_df = cda_mapping_df[cda_mapping_df.node_handle_2 != "NOT APPLICABLE"]

# move special cases (i.e., fields w/o 1-1 mapping) to another df for curation
cda_mapping_other_df = cda_mapping_df[cda_mapping_df["node_handle_2"] == cda_mapping_df["prop_handle_2"]]
cda_mapping_df = cda_mapping_df.drop(cda_mapping_other_df.index) # type: ignore

#cda_mapping_other_df.reset_index(inplace=True)
#cda_mapping_df.reset_index(inplace=True)

# clean up prop_handle_1 identifier fields
cda_mapping_df["prop_handle_1"].mask(cda_mapping_df["prop_handle_1"] == "identifier.value",
                                    "identifier", inplace=True)

# file off prop handle 2 from node handle 2
cda_mapping_df["node_handle_2"] = cda_mapping_df["node_handle_2"].str.rsplit(".", n=1).str[0]
cda_mapping_df["node_handle_2"] = cda_mapping_df["node_handle_2"].str.split(".")

# match MDB formatting for node handles
cda_node_handles = ["Sub"]

In [5]:
output_path = input_path
output_file = input_file.split(".")[0] + "_clean.csv"
output_other_file = input_file.split(".")[0] + "_other.csv"
output_file_path = output_path + output_file
output_other_file_path = output_path + output_other_file

In [ ]:
cda_mapping_df.to_csv(output_file_path, index=False)
cda_mapping_other_df.to_csv(output_other_file_path, index=False)

In [6]:
prop_syn_df = pd.read_csv(output_file_path)
prop_syn_df.head()

,model_1,prop_handle_1,node_handle_1,model_2,prop_handle_2,node_handle_2
0,CDA,id,subject,GDC,submitter_id,"['files', 'cases']"
1,CDA,identifier,subject,GDC,submitter_id,"['files', 'cases']"
2,CDA,sex,subject,GDC,gender,"['files', 'cases', 'demographic']"
3,CDA,race,subject,GDC,race,"['files', 'cases', 'demographic']"
4,CDA,ethnicity,subject,GDC,ethnicity,"['files', 'cases', 'demographic']"


In [7]:
def link_prop_syn_csv(
    mdbn: NelsonMDB,
    csv_path: str,
    #entity_type: str,
    add_missing_ent: bool = False
    ) -> None:
    """Given a CSV of syonymous Properties, links each via a Concept node"""    
    
    model_node_translator = {
        "files": "file",
        "cases": "case",
        "demographics": "demographic",
        "projects": "project",
        "diagnoses": "diagnosis",
        "treatments": "treatment",
        "samples": "sample"
    }
    cda_node_translator = {
        "subject": "Subject",
        "researchsubject": "ResearchSubject",
        "diagnosis": "Diagnosis",
        "treatment": "Treatment",
        "specimen": "Specimen",
        "file": "File"
    }

    with open(csv_path, encoding="UTF-8") as csvfile:
        syn_reader = csv.DictReader(csvfile)
        for row in syn_reader:            
            # convert node handles to match names in MDB
            node_handle_2_list = ast.literal_eval(row["node_handle_2"])
            translated_n2_handles = []
            for handle in node_handle_2_list:
                if handle in model_node_translator.keys():
                    new_handle = model_node_translator[handle]
                    translated_n2_handles.append(new_handle)
                else:
                    translated_n2_handles.append(handle)
            node_handle_1 = cda_node_translator[row["node_handle_1"]]
            
            # generate bento-meta objects for entiies in row
            prop_1 = Property({
                "model": row["model_1"],
                "handle": row["prop_handle_1"]
            })
            node_1 = Node({
                "model": row["model_1"],
                "handle": node_handle_1
            })
            prop_2 = Property({
                "model": row["model_2"],
                "handle": row["prop_handle_2"]
            })
            node_2 = Node({
                "model": row["model_2"],
                "handle": translated_n2_handles[-1]
            })

            # add nanoid to properties so that they can be uniquely identified
            # checking if a nanoid exists for the (node)-[:has_property]->(property) triple
            prop_1.nanoid = mdbn.get_or_make_nano(prop_1, node_1.handle) #type: ignore
            node_1.nanoid = mdbn.get_or_make_nano(node_1)
            prop_2.nanoid = mdbn.get_or_make_nano(prop_2, node_2.handle) #type: ignore
            node_2.nanoid = mdbn.get_or_make_nano(node_2)
            
            row_props = [prop_1, prop_2]
            row_nodes = [node_1, node_2]

            # create properties/nodes if they aren't already in MDB
            row_ents = []
            row_ents.extend(row_props)
            row_ents.extend(row_nodes)
            for ent in row_ents:
                ent_count = mdbn.get_entity_count(ent)[0]                
                if not ent_count and add_missing_ent:
                    mdbn.create_entity(ent)
                else:
                    print(f"{get_entity_type(ent).capitalize()} {ent.handle} in model: {ent.model} already exists with nanoid: {ent.nanoid}")
            
            for i, prop in enumerate(row_props):
                if add_missing_ent:
                    mdbn.create_relationship(                        
                        src_entity=row_nodes[i],
                        dst_entity=prop,
                        relationship="has_property")

            # now that everything is created that needs to be (if applicable), actually link ents via concept
            mdbn.link_synonyms(prop_1, prop_2, add_missing_ent=add_missing_ent)

In [14]:
link_prop_syn_csv(
    mdbn = mdbn,
    csv_path = output_file_path,
    add_missing_ent=True
)

Property id in model: CDA already exists with nanoid: bTzrp7
Property submitter_id in model: GDC already exists with nanoid: wgwT0U
Node Subject in model: CDA already exists with nanoid: MQN9i0
Node case in model: GDC already exists with nanoid: v0Bmxb
prop: id in CDA; node: Subject in CDA
Creating has_property relationship from src node with properties: {handle: 'Subject', model: 'CDA', nanoid: 'MQN9i0'} to dst property with properties: {handle: 'id', model: 'CDA', nanoid: 'bTzrp7'}
prop: submitter_id in GDC; node: case in GDC
Creating has_property relationship from src node with properties: {handle: 'case', model: 'GDC', nanoid: 'v0Bmxb'} to dst property with properties: {handle: 'submitter_id', model: 'GDC', nanoid: 'wgwT0U'}
Both terms are already connected via Concept VSjr1V
Property identifier in model: CDA already exists with nanoid: 8UQgCE
Property submitter_id in model: GDC already exists with nanoid: wgwT0U
Node Subject in model: CDA already exists with nanoid: MQN9i0
Node ca

In [ ]:
# getting any extra nodes leading to property to connect later
extra_node_2s = []
for node_handle in translated_n2_handles[:-1]:
    node = Node({
        "model": row["model_2"],
        "handle": node_handle
    })
    extra_node_2s.append(node)

# add pattern to connect extra_node_2s and node_2 if necessary
for i, node in enumerate(translated_n2_handles[:-1]):
    next_node = translated_n2_handles[i+1]
    if add_missing_ent and len(translated_n2_handles) > 1:
        describes_rel = Edge({
            "handle": "describes",
            "model": row["model_2"]
            })
        rel_nano_list = mdbn.get_entity_nano(
            describes_rel,
            extra_handle_1=node,
            extra_handle_2=next_node)
        if rel_nano_list: 
            describes_rel.nanoid = rel_nano_list[0]
        else:
            describes_rel.nanoid = mdbn.make_nano()
        mdbn.create_relationship(
            src_entity=describes_rel,
            dst_entity=node,
            relationship="has_src")
        mdbn.create_relationship(
            src_entity=describes_rel,
            dst_entity=next_node,
            relationship="has_dst")

In [17]:
pd.read_csv(output_other_file_path).head()

,model_1,prop_handle_1,node_handle_1,model_2,prop_handle_2,node_handle_2
0,CDA,identifier.system,subject,GDC,GDC,GDC
1,CDA,species,subject,GDC,Homo sapiens,Homo sapiens
2,CDA,identifier.system,subject,PDC,PDC,PDC
3,CDA,identifier.system,subject,IDC,IDC,IDC
4,CDA,identifier.system,researchsubject,GDC,GDC,GDC


In [25]:
mdbn.get_entity_nano(
    entity=Property({"handle": "id", "model": "CDA", }),
    extra_handle_1="Subject"
)

[]